In [ ]:
# Version requirements: Python 3.14.2, numpy 2.3.5, matplotlib 3.10.8, pywt 1.8.0, scipy 1.16.3
import numpy as np
import matplotlib.pyplot as plt
import pywt

In [ ]:
# Define your functions
def gaussian(x, x0, sigma):
    return np.exp(-np.power((x - x0) / sigma, 2.0) / 2.0)

# Create chirp signal, which is the sum of two signals with time-dependent frequencies
def make_chirp(t, t0, a):
    frequency = (a * (t + t0)) ** 2
    chirp = np.sin(2 * np.pi * frequency * t)
    return chirp, frequency

# Generate signal
time = np.linspace(0, 1, 8048)
chirp1, frequency1 = make_chirp(time, 0.2, 9)
chirp2, frequency2 = make_chirp(time, 0.1, 5)
chirp = chirp1 + 0.6 * chirp2
chirp *= gaussian(time, 0.5, 0.2)
# Plot signal and frequency components
fig, axs = plt.subplots(2, 1, sharex=True, figsize=(12,6))
axs[0].plot(time, chirp)
axs[0].set_ylabel("Signal amplitude")
axs[0].set_title("Input signal")

axs[1].plot(time, frequency1, label="Chirp1 frequency")
axs[1].plot(time, frequency2, label="Chirp2 frequency")
axs[1].set_yscale("log")
axs[1].set_xlabel("Time (s)")
axs[1].set_ylabel("Frequency (Hz)")
axs[1].legend()
axs[1].set_title("Time-dependent frequencies")

plt.tight_layout()
plt.show()

In [ ]:
signal = (chirp1+0.6*chirp2) * gaussian(time, 0.5, 0.2)
print("Type of signal:", type(signal))
# Add noises
np.random.seed(0)
noisy_signal = signal + 0.5 * np.random.normal(size=signal.shape)

# Plot the signal
fig, axs = plt.subplots(2, 1, sharex=True, figsize=(12,8))
axs[0].plot(time, signal, label='Original Signal')
axs[0].set_title('Original Signal')
axs[1].plot(time, noisy_signal, label='Noisy Signal', color='orange')
axs[1].set_title('Noisy Signal')
plt.tight_layout()
plt.show()

In [ ]:
def wavelet_denoise(signal, wavelet_name = 'db4', level=5, threshold=0.2):
    coeffs = pywt.wavedec(signal, wavelet_name, level=level)
    # Apply thresholding to detail coefficients
    coeffs_thresholded = [coeffs[0]] + [pywt.threshold(c, threshold, mode='soft') for c in coeffs[1:]]
    # Reconstruct the signal
    denoised_signal = pywt.waverec(coeffs_thresholded, wavelet_name)
    return denoised_signal
